### Before you start
Write down the following variables:

In [ ]:
TENANT_ID="<your tenant id>"
SUBSCRIPTION_ID="<your subscription id>"

**Save your environment ID**: To avoid collision with other participants, we will use the environment name as a suffix for the resource names. You can use your name or any other unique identifier.
For example your initials and a random number: `ed42`

In [ ]:
ENV_NAME="demo01"

We will use ip whitlisting to secure your app.

In [ ]:
WHITELIST_IP=!curl -s ipinfo.io/ip
print(f"We will whitelist your IP address: {WHITELIST_IP}")

**Login with the newly created account**:

In [ ]:
!az login --tenant $TENANT_ID

**Switch to your project subscription ID**:

In [ ]:
!az account set --subscription $SUBSCRIPTION_ID

**Find your Principal ID**:
This will be the user that indexes the documents. In this case, we assume it's run locally, thus we use the signed in user.

In [ ]:
PRINCIPAL_ID_1=!az ad signed-in-user show --only-show-errors --output=tsv --query=objectId
# or
PRINCIPAL_ID_2=!az ad signed-in-user show --only-show-errors --output=tsv --query=id
PRINCIPAL_ID = PRINCIPAL_ID_1[0] if PRINCIPAL_ID_1 else PRINCIPAL_ID_2[0]

print(f"Your principal id is: {PRINCIPAL_ID}")

### Deploying Infrastructure Resources

In [ ]:
!PROJECT=workshop-oai \
AZURE_SUBSCRIPTION_ID=$SUBSCRIPTION_ID \
AZURE_ENV_NAME=$ENV_NAME \
AZURE_LOCATION=eastus \
AZURE_PRINCIPAL_ID=$PRINCIPAL_ID \
AZURE_AUTH_CLIENT_ID="" \
APP_WHITELIST_IP="185.89.153.26/32" \
bash scripts/deploy_infrastructure.sh

In [ ]:
RESOURCE_GROUP_NAME=f"workshop-oai-{ENV_NAME}-rg"
print(f"Your resource group name is: {RESOURCE_GROUP_NAME}")

### Deploying the App Code
Deploy the app code to Azure App Service.

In [ ]:
BACKEND_APP_NAME=!az deployment group show -g $RESOURCE_GROUP_NAME -n web --query properties.outputs.name.value
BACKEND_APP_NAME = BACKEND_APP_NAME[0]

print(f"Your backend app name is: {BACKEND_APP_NAME}")

Upload the code to Azure App Service:

In [ ]:
!az webapp up \
--runtime PYTHON:3.10 \
--sku B1 \
--name $BACKEND_APP_NAME \
--resource-group $RESOURCE_GROUP_NAME \
--subscription $SUBSCRIPTION_ID

## Index the documents
Run the following command to create `.env` file. 

In [ ]:
!bash scripts/create_env.sh $RESOURCE_GROUP_NAME

Install python dependencies:

In [ ]:
!pip install -r requirements.txt

Index the documents:

In [ ]:

!python scripts/indexing.py --throttle 1 --documents-path=data/transformers_docs_medium

## Test the app
Open the app url in the browser and ask a question about transformers library.

In [ ]:
BACKEND_URL = !az deployment group show -g $RESOURCE_GROUP_NAME -n web --query properties.outputs.uri.value
BACKEND_URL = BACKEND_URL[0]

print(f"Your backend url is: {BACKEND_URL}")